# Multicut

Simple example for obtaining the multicut segmentation for the dataset, we have initialized in *init*.

Note that for applying the pipeline to 3d data, there need only to be a few changes:

* Start from an extended 3d supervoxel oversegmentation instead of flat superpixel.
* Set the degree of anisotropy to 1.
* Set use_2d to False.

In [1]:
import os
import numpy as np
import vigra
# rand index for evaluating the multicut score
# (I usually use the implementation from skneuro, but don't want to depend
# on this here, so we use the sklearn implementation)
# note that this doesnt have an ignore label...
from sklearn.metrics import adjusted_rand_score as rand_index
# imports from Neurocut
from MetaSet import MetaSet
from MCSolver import multicut_workflow
from ExperimentSettings import ExperimentSettings
from Postprocessing import merge_small_segments

# get the metaset from the correct cache folder
#cache_folder = "/path/to/cache"
cache_folder = "/home/consti/Work/data_master/cache_neurocut/cache_examples"
meta = MetaSet(cache_folder)

In [2]:
# reload the dataset
meta.load()
ds = meta.get_dataset("dataset")

# id of the segmentation (== superpixels) we will calculate everything for
# we have added only one segmentation, so we have to use 0 here
seg_id = 0

# ExperimentSettings holds all relveant options for the experiments
# they are initialised to sensible defaults and 
# we only have to change a few
mc_params = ExperimentSettings()

# cache folder for the RF
mc_params.set_rfcache(os.path.join(cache_folder, "rf_cache"))
# train RF with 500 trees
mc_params.set_ntrees(500)
# degree of anisotropy for the filter calculation
# (values bigger than 20 lead to calculation in 2d)
# set to 1. for isotropic data (default value)
mc_params.set_anisotropy(25.)
# flag to indicate whether special z - edge features are computed
# set to false for isotropic data (default value)
mc_params.set_use2d(True)

# otherwise, the default parameter should be ok

# list of features taken into account
# "raw" -> filters on raw data accumulated over the edges
# "prob" -> filters on probability maps accumulated over the edges
# "reg" -> region statistics, mapped to the edges
# "topo" -> topology features for the edges
feat_list = ("raw", "prob", "reg", "topo")

# in multicut_from_rf_gt, first a rf is learned with labels from
# groundtruth projections from the training dataset
# then this rf is applied to the test dataset and the resulting 
# probabilites for the edges are converted to energies
# with this energies, the multicut is solved

# note that we have not done a train / test split yet,
# so train and test are the same

# first parameter is the dataset we train the rf on
# second parameter is the dataset we run the multicut on, with weights from the rf
# third and fourth parameter: id of segmentationf for train and test dataset
# fifth in six: features for train and test
# seventh: the parameter object
mc_nodes, mc_edges, mc_energy, t_inf = multicut_workflow(
    ds, ds,
    seg_id, seg_id,
    feat_list, mc_params)
# mc_nodes = result for the segments
# mc_edges = result for the edges
# mc_energy = energy of the solution
# t_inf = time the inference of the mc took

# project the result back to the volume
mc_seg = ds.project_mc_result(seg_id, mc_nodes)
# merge small segments
mc_seg = merge_small_segments(mc_seg, 100)

Computing RAG for seg_id 0
WRITING IN  /home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/rag_seg0.h5 rag
Running multicut on dataset Weights learned on dataset
with solver opengm_exact
Learning random forest with 200 trees
Computing:  edge_gt with args:
(0,)
Loading RAG for seg_id 0 from HDF5:
/home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/rag_seg0.h5
rag
Loading RAG for seg_id 0 from HDF5:
/home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/rag_seg0.h5
rag
Computing:  _adjacent_segments with args:
(0,)
Getting segments adjacent to edges from RAG:
Loading RAG for seg_id 0 from HDF5:
/home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/rag_seg0.h5
rag
Writing result in  /home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/_adjacent_segments_0.h5 data
Writing result in  /home/consti/Work/data_master/cache_neurocut/cache_examples/dataset/edge_gt_0.h5 data
Computing:  ignore2ignorers with args:
(0,)
Loadin

AttributeError: 'module' object has no attribute 'Multicut'

In [ ]:
# evaluate the result
gt = ds.gt()

# evaluation with proper skneuro randIndex
"""
from skneuro.learning import randIndex

gt = gt.astype(np.uint32)
mc_seg = mc_seg.astype(np.uint32)

print randIndex(gt.flatten(), mc_seg.flatten(), ignoreDefaultLabel = True)
"""

ri = rand_index(gt.ravel(), mc_seg.ravel())
print "Multicut Segmentation has a RI of", ri

These numbers don't look too good, but I think this is due to the sklearn metric. Visually, the result does look good and evaluation with the RandIndex from skneuro yields a value of 0.9998.